In [1]:
#Enables extensions for gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

output_data_file = "output_data/cities.csv"

#Downloads csv file
cities_download = output_data_file
cities_data = pd.read_csv(cities_download)

#Puts csv info into a dataframe 
city_weather_data = pd.DataFrame(cities_data)
city_weather_data

,Unnamed: 0,City,Lats,Lngs,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
0,0,Heraclea,37.5133,34.0467,80.31,2,40,3.47,TR,1659643832
1,1,Torbay,47.6666,-52.7314,77.76,100,79,4.00,CA,1659643534
2,2,Nyrob,60.7330,56.7202,64.44,62,74,2.48,RU,1659643833
3,3,Clyde River,70.4692,-68.5914,42.03,100,83,5.14,CA,1659643834
4,4,Bengkulu,-3.8004,102.2655,73.85,97,90,2.98,ID,1659643834
...,...,...,...,...,...,...,...,...,...,...
560,560,Tazovsky,67.4667,78.7000,58.21,100,64,10.38,RU,1659644115
561,561,Buenos Aires,-34.6132,-58.3772,53.98,0,61,4.61,AR,1659644116
562,562,Nabire,-3.3667,135.4833,75.51,95,92,2.53,ID,1659644116
563,563,Pareora,-44.4871,171.2132,45.18,100,76,8.21,NZ,1659644117


In [3]:
#Creates a google map based on the data in the dataframe 
gmaps.configure(api_key=g_key)

Locations = city_weather_data[['Lats','Lngs']]
weight = city_weather_data['Humidity']
figure_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(Locations, weights=weight, dissipating=False, max_intensity=100, point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
#Filters and creates a new dataframe based on the following parameters
city_filter = city_weather_data.loc[(city_weather_data['Max Temp'] >= 70) & 
                      (city_weather_data['Max Temp'] <= 80) & 
                      (city_weather_data['Wind Speed'] < 10) &
                      (city_weather_data['Cloudiness'] == 0)]
city_filter_df = city_filter.reset_index(inplace= False)
city_filter_df

,index,Unnamed: 0,City,Lats,Lngs,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
0,9,9,Orbetello,42.4408,11.2196,79.86,0,79,6.02,IT,1659643837
1,48,48,Arraial do Cabo,-22.9661,-42.0278,75.06,0,78,8.05,BR,1659643855
2,105,105,Shitanjing,39.2342,106.3439,76.28,0,67,3.80,CN,1659643884
3,117,117,Behshahr,36.6923,53.5526,79.41,0,66,5.91,IR,1659643891
4,156,156,San Fernando del Valle de Catamarca,-28.4696,-65.7852,70.77,0,35,1.01,AR,1659643910
5,265,265,Marand,38.4329,45.7749,75.20,0,47,6.80,IR,1659643965
6,279,279,Lasem,-6.6922,111.4527,72.63,0,78,8.23,ID,1659643972
7,295,295,Qazvin,36.2797,50.0049,75.29,0,52,9.93,IR,1659643980
8,325,325,Al Bardīyah,31.7561,25.0865,78.89,0,70,8.41,LY,1659643995
9,327,327,Nagato,34.3833,131.2000,79.47,0,86,8.50,JP,1659643996


In [5]:
# city_filter_df[city_filter_df.loc[0, "Lats"]]

# Lats_par = city_filter_df.loc[i, "Lats"]
# Lngs_par = city_filter_df.loc[i, "Lngs"]

In [6]:
#Creates a list of nearby hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_dict = {
    "Hotel Name": [],
}
for i in range(len(city_filter_df)):
    Lats_par = city_filter_df.loc[i, "Lats"]
    Lngs_par = city_filter_df.loc[i, "Lngs"]
    
    params = {
        "location": f"{Lats_par}, {Lngs_par}" ,
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }

# base url


# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        response = response.json()
        hotel_dict["Hotel Name"].append(response['results'][0]['name'])
#         hotel_dict["Hotel Name"].append(response['results']['name'])
#response
#     hotel_df = city_filter_df.assign(Hotel Name=[])
#     hotal_df
hotel_dict

{'Hotel Name': ['Argentario Golf & Wellness Resort',
  'Pousada Porto Praia',
  'Yingbin Hotel',
  'Abbas Abad Hotel & Restaurant',
  'Amerian Catamarca Park Hotel',
  'کوچه صفا',
  'Nyah Lasem | Museum, Warung & Community Center',
  'MarMar Hotel',
  'بورتو بردية',
  'いそう庵',
  'Foyer ADM',
  'Erenhot Rurouni Youth Hostel',
  'WB APARTMENT',
  'Los Mosaicos',
  'Hua Ling Grand Hotel Urumqi',
  'Catering Chotěboř s.r.o.']}

In [12]:
#Creates a merged dataframe for the hotels
hname_df = pd.DataFrame(hotel_dict)
x = [hname_df, city_filter_df]
hotel_df = pd.concat(x, axis = 1)
#hotel_df.drop('Index', 'Unnamed:0', inplace=True, axis=0)
#hotel_df.pop('Index', 'Unnamed:0')
hotel_df

,Hotel Name,index,Unnamed: 0,City,Lats,Lngs,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
0,Argentario Golf & Wellness Resort,9,9,Orbetello,42.4408,11.2196,79.86,0,79,6.02,IT,1659643837
1,Pousada Porto Praia,48,48,Arraial do Cabo,-22.9661,-42.0278,75.06,0,78,8.05,BR,1659643855
2,Yingbin Hotel,105,105,Shitanjing,39.2342,106.3439,76.28,0,67,3.80,CN,1659643884
3,Abbas Abad Hotel & Restaurant,117,117,Behshahr,36.6923,53.5526,79.41,0,66,5.91,IR,1659643891
4,Amerian Catamarca Park Hotel,156,156,San Fernando del Valle de Catamarca,-28.4696,-65.7852,70.77,0,35,1.01,AR,1659643910
5,کوچه صفا,265,265,Marand,38.4329,45.7749,75.20,0,47,6.80,IR,1659643965
6,"Nyah Lasem | Museum, Warung & Community Center",279,279,Lasem,-6.6922,111.4527,72.63,0,78,8.23,ID,1659643972
7,MarMar Hotel,295,295,Qazvin,36.2797,50.0049,75.29,0,52,9.93,IR,1659643980
8,بورتو بردية,325,325,Al Bardīyah,31.7561,25.0865,78.89,0,70,8.41,LY,1659643995
9,いそう庵,327,327,Nagato,34.3833,131.2000,79.47,0,86,8.50,JP,1659643996


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lats", "Lngs"]]
locations

,Lats,Lngs
0,42.4408,11.2196
1,-22.9661,-42.0278
2,39.2342,106.3439
3,36.6923,53.5526
4,-28.4696,-65.7852
5,38.4329,45.7749
6,-6.6922,111.4527
7,36.2797,50.0049
8,31.7561,25.0865
9,34.3833,131.2000


In [9]:
#Prints a google map based on the hotel dataframe

#locations.values.tolist()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…